Lab: 最大概似估計
================

In [1]:
import torch

In [2]:
from torch.distributions import Normal
normal = Normal(loc=0., scale=1.)

In [3]:
print("random sample with shape ():\n",
      normal.sample())
print("random sample with shape (3,):\n",
      normal.sample(sample_shape=(3,)))
print("random sample with shape (2,3):\n",
      normal.sample(sample_shape=(2, 3)))

random sample with shape ():
 tensor(-2.2001)
random sample with shape (3,):
 tensor([-0.6555, -0.1522, -0.1238])
random sample with shape (2,3):
 tensor([[ 1.0799,  1.3602, -1.3700],
        [ 1.5724,  0.0683,  0.8137]])


In [4]:
print("cumulative probability given value with shape ():\n",
      normal.cdf(value=0), "\n")
print("cumulative probability given value with (3,):\n",
      normal.cdf(value=torch.Tensor([-1, 0, .5])), "\n")
print("cumulative probability given value with (2,3):\n",
      normal.cdf(value=torch.Tensor([[-1, 0, .5], [-2, 1, 3]])))

cumulative probability given value with shape ():
 tensor(0.5000) 

cumulative probability given value with (3,):
 tensor([0.1587, 0.5000, 0.6915]) 

cumulative probability given value with (2,3):
 tensor([[0.1587, 0.5000, 0.6915],
        [0.0228, 0.8413, 0.9987]])


In [5]:
print("cumulative probability given value with shape ():\n",
      normal.log_prob(value=0), "\n")
print("cumulative probability given value with (3,):\n",
      normal.log_prob(value=torch.Tensor([-1, 0, .5])), "\n")
print("cumulative probability given value with (2,3):\n",
      normal.log_prob(value=torch.Tensor([[-1, 0, .5], [-2, 1, 3]])))

cumulative probability given value with shape ():
 tensor(-0.9189) 

cumulative probability given value with (3,):
 tensor([-1.4189, -0.9189, -1.0439]) 

cumulative probability given value with (2,3):
 tensor([[-1.4189, -0.9189, -1.0439],
        [-2.9189, -1.4189, -5.4189]])


In [6]:
print(normal)

Normal(loc: 0.0, scale: 1.0)


In [7]:
print(normal.batch_shape)
print(normal.event_shape)

torch.Size([])
torch.Size([])


In [8]:
normal_batch = Normal(loc=torch.Tensor([0., 1.]), scale=torch.Tensor([1., 1.5]))
print(normal_batch)

Normal(loc: torch.Size([2]), scale: torch.Size([2]))


In [9]:
print("random sample with sample_shape ():\n",
      normal_batch.sample(), "\n")
print("random sample with sample_shape (3,):\n",
      normal_batch.sample(sample_shape=(3,)), "\n")
print("random sample with sample_shape (2,3):\n",
      normal_batch.sample(sample_shape=(2,3)))


random sample with sample_shape ():
 tensor([0.2274, 0.0266]) 

random sample with sample_shape (3,):
 tensor([[-1.3639,  3.5289],
        [-1.1314,  0.4441],
        [-1.7837,  0.5650]]) 

random sample with sample_shape (2,3):
 tensor([[[-1.1421,  0.6511],
         [-0.7792, -0.0861],
         [ 0.1006,  2.2322]],

        [[ 0.2150,  0.0298],
         [ 1.1210,  0.3860],
         [ 0.2337, -0.3509]]])


In [10]:
print("log-probability given value with shape ():\n",
      normal_batch.log_prob(0), "\n")
print("log-probability given value with shape (2,):\n",
      normal_batch.log_prob(torch.Tensor([0, 0])), "\n")
print("log-probability given value with shape (2,1):\n",
      normal_batch.log_prob(torch.Tensor([[0], [0]])))

log-probability given value with shape ():
 tensor([-0.9189, -1.5466]) 

log-probability given value with shape (2,):
 tensor([-0.9189, -1.5466]) 

log-probability given value with shape (2,1):
 tensor([[-0.9189, -1.5466],
        [-0.9189, -1.5466]])


In [11]:
from torch.distributions import MultivariateNormal
mvn = MultivariateNormal(
    loc=torch.Tensor([0, 1]),
    scale_tril=torch.cholesky(torch.Tensor([[1., 0.], [0., .5]])))
print(mvn)


MultivariateNormal(loc: torch.Size([2]), scale_tril: torch.Size([2, 2]))


In [12]:
print("random sample with sample_shape ():\n",
      mvn.sample(), "\n")
print("random sample with sample_shape (3,):\n",
      mvn.sample(sample_shape=(3,)), "\n")
print("random sample with sample_shape (2, 3):\n",
      mvn.sample(sample_shape=(2, 3)))

random sample with sample_shape ():
 tensor([-0.3706,  0.9224]) 

random sample with sample_shape (3,):
 tensor([[ 0.0142, -0.2571],
        [ 0.2588,  0.9381],
        [-0.4774,  1.0325]]) 

random sample with sample_shape (2, 3):
 tensor([[[ 1.3255,  1.3830],
         [ 1.7761,  2.1592],
         [-1.7575,  0.8117]],

        [[-1.6518,  1.7633],
         [ 0.8681,  0.9253],
         [ 0.9465,  2.3077]]])


In [13]:
print("log-probability given value with shape (2,):\n",
      mvn.log_prob(torch.Tensor([0, 0])), "\n")
print("log-probability given value with shape (2,1):\n",
      mvn.log_prob(torch.Tensor([[0, 0], [0, 0]])))

log-probability given value with shape (2,):
 tensor(-2.4913) 

log-probability given value with shape (2,1):
 tensor([-2.4913, -2.4913])


In [14]:
from torch.distributions import Independent
normal_batch = Independent(normal_batch, reinterpreted_batch_ndims=1)
print(normal_batch.batch_shape)
print(normal_batch.event_shape)


torch.Size([])
torch.Size([2])


In [15]:
mvn_batch = MultivariateNormal(
    loc=torch.Tensor([[0, 1],[1, 2],[2, 3]]),
    scale_tril=torch.cholesky(torch.Tensor([[1., .2], [.2, .5]])))
mvn_batch

MultivariateNormal(loc: torch.Size([3, 2]), scale_tril: torch.Size([3, 2, 2]))

In [16]:
print("random sample with sample_shape ():\n",
      mvn_batch.sample(), "\n")
print("random sample with sample_shape (3,):\n",
      mvn_batch.sample(sample_shape=(3,)), "\n")
print("random sample with sample_shape (2, 3):\n",
      mvn_batch.sample(sample_shape=(2, 3)))


random sample with sample_shape ():
 tensor([[-0.1869,  0.7251],
        [ 1.0426,  2.2433],
        [ 1.7472,  3.3224]]) 

random sample with sample_shape (3,):
 tensor([[[-0.2009,  0.2792],
         [ 1.1694,  1.4674],
         [ 1.1404,  3.6818]],

        [[-0.9680,  1.2411],
         [-0.0784,  0.6412],
         [ 2.6169,  2.9247]],

        [[ 1.0264,  1.4182],
         [ 2.6512,  2.5538],
         [ 2.4851,  3.2090]]]) 

random sample with sample_shape (2, 3):
 tensor([[[[-0.6230,  0.3428],
          [ 1.3693,  1.6382],
          [ 2.3221,  3.1609]],

         [[-0.5281,  1.1079],
          [ 3.9142,  3.3278],
          [ 1.6654,  2.7964]],

         [[-0.4935,  0.3863],
          [ 1.7580,  2.5757],
          [ 1.7618,  3.1079]]],


        [[[ 0.7393,  0.3040],
          [ 0.7228,  2.2151],
          [ 1.7926,  2.4240]],

         [[-0.8338,  0.3145],
          [ 1.0488,  2.3009],
          [ 2.9569,  4.2369]],

         [[ 0.2136,  0.6858],
          [ 1.9889,  2.5046],
     

In [17]:
mu_true = torch.tensor([5.])
sigma_true = torch.tensor([2.])
normal_model_true = Normal(
    loc=mu_true,
    scale=sigma_true)
print("normal model:\n", normal_model_true, "\n")

normal model:
 Normal(loc: tensor([5.]), scale: tensor([2.])) 



In [18]:
sample_size = 1000
x = normal_model_true.sample(sample_shape=(sample_size,))
loss_value = -torch.mean(torch.sum(normal_model_true.log_prob(x), dim = 1))
print("negative likelihood value is", loss_value)


negative likelihood value is tensor(2.1018)


In [19]:
epochs = 200
lr = 1.0
mu = torch.tensor([0.], requires_grad=True)
sigma = torch.tensor([1.], requires_grad=True)
optimizer = torch.optim.Adam([mu, sigma], lr=.5)
for epoch in range(epochs):
    normal_model = Normal(loc=mu, scale=sigma)
    loss_value = -torch.mean(normal_model.log_prob(x))
    optimizer.zero_grad()
    loss_value.backward() # compute the gradient
    optimizer.step()

In [20]:
print("ML mean by gradient descent:", mu)
print("ML std by gradient descent:", sigma)

ML mean by gradient descent: tensor([5.0841], requires_grad=True)
ML std by gradient descent: tensor([1.9769], requires_grad=True)


In [21]:
print("ML mean by formula:", torch.mean(x))
print("ML std by formula:", torch.std(x, unbiased=False))

ML mean by formula: tensor(5.0841)
ML std by formula: tensor(1.9776)


In [22]:
mu_true = torch.tensor([-1., 0., 1.])
sigma_tril_true = torch.tensor([[3., 0., 0.], [2., 1., 0.], [.4, .5, .5]])
mvn_model_true = MultivariateNormal(
    loc=mu_true,
    scale_tril=sigma_tril_true)
print("true mean vector: \n", mvn_model_true.mean)
print("true covariance matrix: \n", mvn_model_true.covariance_matrix)

true mean vector: 
 tensor([-1.,  0.,  1.])
true covariance matrix: 
 

tensor([[9.0000, 6.0000, 1.2000],
        [6.0000, 5.0000, 1.3000],
        [1.2000, 1.3000, 0.6600]])


In [23]:
sample_size = 1000
x = mvn_model_true.sample(sample_shape=(sample_size,))
loss_value = -torch.mean(mvn_model_true.log_prob(x))
print("negative likelihood value is", loss_value)


negative likelihood value is tensor(4.7174)


In [24]:
epochs = 500
lr = .1
mu = torch.tensor(
    [0., 0., 0.], requires_grad=True)
sigma_tril = torch.tensor(
    [[1., 0., 0.], [0., 1., 0.], [0., 0., 1.]],
    requires_grad=True)
optimizer = torch.optim.Adam([mu, sigma_tril], lr=lr)

for epoch in range(epochs):
    mvn_model = MultivariateNormal(
    loc=mu,
    scale_tril=sigma_tril)
    loss_value = -torch.mean(mvn_model.log_prob(x))
    optimizer.zero_grad()
    loss_value.backward() # compute the gradient
    optimizer.step()

In [25]:
print("ML mean by gradient descent: \n",
      mu)
print("ML covariance by gradient descent: \n",
      sigma_tril @ torch.transpose(sigma_tril, 0, 1))

ML mean by gradient descent: 
 tensor([-0.9634, -0.0140,  0.9845], requires_grad=True)
ML covariance by gradient descent: 
 tensor([[9.0569, 6.1244, 1.2179],
        [6.1244, 5.2158, 1.3481],
        [1.2179, 1.3481, 0.6764]], grad_fn=<MmBackward>)


In [26]:
sample_mean = torch.mean(x, dim = 0)
sample_moment2 = (torch.transpose(x, 0, 1) @ x) / sample_size
sample_cov = sample_moment2 - torch.ger(sample_mean, sample_mean)
print("ML mean by formula: \n",
      sample_mean)
print("ML covariance by formula: \n",
      sample_cov)

ML mean by formula: 
 tensor([-0.9634, -0.0140,  0.9845])
ML covariance by formula: 
 tensor([[9.0569, 6.1244, 1.2179],
        [6.1244, 5.2158, 1.3481],
        [1.2179, 1.3481, 0.6764]])
